In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 9 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Equation de la chaleur - visio EDO à discrétisation spatiale fixée
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Equation de la chaleur - visio EDO à discrétisation spatiale fixée

In [ ]:
from dataclasses import dataclass
import numpy as np
from scipy.integrate import solve_ivp
from scipy.sparse import diags
from scipy import sparse
from scipy.sparse.linalg import spsolve
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

## Equation de la chaleur

On considère l'équation de la chaleur dans le cas 1D :

$$
\left\{
\begin{aligned}
& \partial_t u(x,t) - D \, \partial_{xx} u(x,t) = 0 \;\; \text{pour} \; -L < x < L\\
& \partial_{x} u(-L,t) = \partial_{x} u(L,t) = 0,\\
& u(x,t=t_0)  =  \frac{1}{2 \sqrt{\pi D t_0}} \exp \Bigg(-\frac{x^2}{\sqrt{4Dt_0}} \Bigg),
\end{aligned}
\right.
$$

où la donnée initiale est donc la solution élémentaire de l'équation de la chaleur au temps $t_0$ et 
avec des conditions aux limites de type Neumann homogène.

Pour la simulation nous prenons $L=5$, $t_0=0.01$ et $D=1$. Le temps final est pris à $t_0+T$ avec $T=0.1$. Nous prenons aussi 1001 points de maillage en espace, discrétisation qui restera fixée pour cette étude dans l'esprit EDO. On pourra consulter l'autre notebook pour l'analyse au sens des EDP où le pas en espace tend aussi vers zéro.

In [ ]:
#############################################
class heat_model:

    def __init__(self, d, xmin, xmax, nx) :
        self.d = d
        self.xmin = xmin
        self.xmax = xmax
        self.nx = nx
        self.dx = (xmax-xmin)/(nx-1)
        self.oneoverdxdx = 1/(self.dx*self.dx)
        
        doverdxdx = d/(self.dx**2)
        diag = np.repeat(-2, nx)
        sub_diag = np.repeat([1., 2.], [nx-2, 1])
        self.a = doverdxdx * diags([diag, sub_diag, np.flip(sub_diag)], [0, -1, 1])

    def fcn(self, t, y):
        return self.a.dot(y)
        #dy = np.zeros_like(y)
        #dy[0]    = self.oneoverdxdx * (-2*y[1] + 2*y[2])
        #dy[1:-1] = self.oneoverdxdx * (y[:-2] - 2*y[1:-1] + y[2:])
        #dy[-1]   = self.oneoverdxdx * (2*y[-2] - 2*y[-1])
        #return dy
    
    def fcn_exact(self, t):
        xmin = self.xmin
        xmax = self.xmax
        d = self.d
        nx = self.nx
        dx = self.dx
        x = np.linspace(xmin, xmax, nx)
        y = (1/(2*np.sqrt(np.pi*t))) * np.exp(-(x*x)/(4.*t))
        return y

## Méthode d'Euler explicite

In [ ]:
@dataclass
class ode_result:
    t: np.ndarray 
    y: np.ndarray    
    
#############################################
def forward_euler(tini, tend, nt, yini, fcn):
    
    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = np.copy(yini)

    for it, tn  in enumerate(t[:-1]):
        yn = yn + dt*fcn(tn, yn)
        
    return ode_result(t, yn)

### Solution

In [ ]:
d = 1.
xmin = -5.
xmax = 5.
nx = 1001
tini = 0.01
tend = 0.11

hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
dx = hm.dx
print('Dicrétisation spatiale : dx = ', dx)
x = np.linspace(xmin, xmax, nx)
fcn_exact = hm.fcn_exact
fcn = hm.fcn

yini = fcn_exact(tini)
yexa = fcn_exact(tend)

sol_qexa = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=[tend], rtol=1e-12, atol=1.e-12)
yqexa = sol_qexa.y[:,0]

fig = make_subplots(rows=2, cols=1, subplot_titles=("Solution", "Erreur"), vertical_spacing=0.15)

fig.add_trace(go.Scattergl(x=x, y=yexa, name=f'sol exacte', legendgroup='sol', line_color='rgb(76,114,176)'), row=1, col=1)
fig.add_trace(go.Scattergl(x=x, y=yqexa, name=f'sol quasi-exacte', mode='lines+markers', legendgroup='sol'), row=1, col=1)

nt_vn = int(2*(tend-tini)/(dx*dx)) + 1
nt = [nt_vn-20, nt_vn, nt_vn+1000, nt_vn+3000, nt_vn+8000]
for nt_i in nt:
    #print('dt = ', (tend-tini)/(nt_i-1))
    sol_num = forward_euler(tini, tend, nt_i, yini, fcn)
    ysol = sol_num.y
    err_exa  = np.abs(yexa-ysol)
    err_qexa = np.abs(yqexa-ysol)
    err_dx   = np.abs(yqexa-yexa)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=ysol, name=f'sol numérique', mode='lines+markers', legendgroup='sol', line_color='rgb(85,168,104)'), row=1, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_exa, name='err calculée à partir <br>  de la solution exacte', mode='lines+markers', legendgroup='err', line_color='rgb(76,114,176)'), row=2, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_qexa, name='err calculée à partir <br>  de la solution quasi-exacte', mode='lines+markers', legendgroup='err', line_color='rgb(221,132,82)'), row=2, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_dx, name='err en dx', mode='lines+markers', legendgroup='err', line_color='rgb(196,78,82)'), row=2, col=1)

for i in range(2+4,6+4): fig.data[i].visible = True

steps = []
for i, nt_i in enumerate(nt):
    args = [{"visible": [(el==0) or (el==1) or (el==4*i+2) or (el==4*i+3) or (el==4*i+4) or (el==4*i+5)  for el in range(len(fig.data))]}]
    #print(args)
    step = dict(method="update", label = f"{nt_i}", args=args)
    steps.append(step)
sliders = [dict(active=1, currentvalue={"prefix": "nt : "}, steps=steps)]

legend = dict(x=0.8, bgcolor='rgba(0,0,0,0)', tracegroupgap=280, groupclick='toggleitem')
fig.update_layout(height=800, sliders=sliders, legend=legend)
fig.update_yaxes(exponentformat='e')
fig.show()

### Ordre

In [ ]:
d = 1.
xmin = -5.
xmax = 5.
nx = 1001
tini = 0.01
tend = 0.11

hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
dx = hm.dx
print('Dicrétisation spatiale : dx = ', dx)
x = np.linspace(xmin, xmax, nx)
fcn_exact = hm.fcn_exact
fcn = hm.fcn

yini = fcn_exact(tini)
yexa = fcn_exact(tend)

l_dt = []
l_err_qexa = [] 

for nt in [2001,  5001, 20001, 200001]:

    dt = (tend-tini)/(nt-1)

    sol = forward_euler(tini, tend, nt, yini, fcn)
    ysol = sol.y
    
    tol = 1.e-6
    t = np.linspace(tini, tend, nt)
    sol_qexa = solve_ivp(fcn, (tini, tend), yini, t_eval=t, method="Radau", rtol=tol, atol=1.e-3*tol)
    yqexa = sol_qexa.y[:,-1]

    # error
    err_qexa = np.abs(yqexa-ysol)
    norm_err_qexa = np.linalg.norm(err_qexa) / np.sqrt(nx)
    #print(f"Norme l2 de l'erreur pour dt = {dt:.3e} : {norm_err_qexa:.10e}")

    l_dt.append(dt)
    l_err_qexa.append(norm_err_qexa)

fig = go.Figure()
fig.add_trace(go.Scatter(x=l_dt, y=l_err_qexa, name='erreur'))
fig.add_trace(go.Scatter(x=l_dt, y=l_dt, name='pente 1'))
fig.update_xaxes(type='log', exponentformat='e', title='dt')
fig.update_yaxes(type='log', exponentformat='e', title='erreur')
fig.update_layout(height=500)
fig.show()

## Méthode de Heun

In [ ]:
#############################################
def heun(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = np.copy(yini)

    for it, tn  in enumerate(t[:-1]):
        k1 = fcn(tn, yn)
        k2 = fcn(tn + dt, yn + dt*k1)
        yn = yn + dt/2*(k1+k2)

    return ode_result(t, yn)

### Solution

In [ ]:
d = 1.
xmin = -5.
xmax = 5.
nx = 1001
tini = 0.01
tend = 0.11

hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
dx = hm.dx
print('Dicrétisation spatiale : dx = ', dx)
x = np.linspace(xmin, xmax, nx)
fcn_exact = hm.fcn_exact
fcn = hm.fcn

yini = fcn_exact(tini)
yexa = fcn_exact(tend)

sol_qexa = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=[tend], rtol=1e-12, atol=1.e-12)
yqexa = sol_qexa.y[:,0]

fig = make_subplots(rows=2, cols=1, subplot_titles=("Solution", "Erreur"), vertical_spacing=0.15)

fig.add_trace(go.Scattergl(x=x, y=yexa, name=f'sol exacte', legendgroup='sol', line_color='rgb(76,114,176)'), row=1, col=1)
fig.add_trace(go.Scattergl(x=x, y=yqexa, name=f'sol quasi-exacte', mode='lines+markers', legendgroup='sol'), row=1, col=1)

nt_vn = int(2*(tend-tini)/(dx*dx)) + 1
nt = [nt_vn-20, nt_vn, nt_vn+1000, nt_vn+3000, nt_vn+8000]
for nt_i in nt:
    sol_num = heun(tini, tend, nt_i, yini, fcn)
    ysol = sol_num.y
    err_exa  = np.abs(yexa-ysol)
    err_qexa = np.abs(yqexa-ysol)
    err_dx   = np.abs(yqexa-yexa)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=ysol, name=f'sol numérique', mode='lines+markers', legendgroup='sol', line_color='rgb(85,168,104)'), row=1, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_exa, name='err calculée à partir <br>  de la solution exacte', mode='lines+markers', legendgroup='err', line_color='rgb(76,114,176)'), row=2, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_qexa, name='err calculée à partir <br>  de la solution quasi-exacte', mode='lines+markers', legendgroup='err', line_color='rgb(221,132,82)'), row=2, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_dx, name='err en dx', mode='lines+markers', legendgroup='err', line_color='rgb(196,78,82)'), row=2, col=1)

for i in range(2+4,6+4): fig.data[i].visible = True

steps = []
for i, nt_i in enumerate(nt):
    args = [{"visible": [(el==0) or (el==1) or (el==4*i+2) or (el==4*i+3) or (el==4*i+4) or (el==4*i+5)  for el in range(len(fig.data))]}]
    #print(args)
    step = dict(method="update", label = f"{nt_i}", args=args)
    steps.append(step)
sliders = [dict(active=1, currentvalue={"prefix": "nt : "}, steps=steps)]

legend = dict(x=0.8, bgcolor='rgba(0,0,0,0)', tracegroupgap=280, groupclick='toggleitem')
fig.update_layout(height=800, sliders=sliders, legend=legend)
fig.update_yaxes(exponentformat='e')
fig.show()

### Ordre

In [ ]:
d = 1.
xmin = -5.
xmax = 5.
nx = 1001
tini = 0.01
tend = 0.11

hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
dx = hm.dx
print('Dicrétisation spatiale : dx = ', dx)
x = np.linspace(xmin, xmax, nx)
fcn_exact = hm.fcn_exact
fcn = hm.fcn

yini = fcn_exact(tini)
yexa = fcn_exact(tend)

l_dt = []
l_err_qexa = [] 

for nt in [2001, 5001, 10001, 20001, 50001, 100001]: 

    dt = (tend-tini)/(nt-1)

    sol = heun(tini, tend, nt, yini, fcn)
    ysol = sol.y
    
    tol = 1.e-9
    t = np.linspace(tini, tend, nt)
    sol_qexa = solve_ivp(fcn, (tini, tend), yini, t_eval=t, method="Radau", rtol=tol, atol=1.e-3*tol)
    yqexa = sol_qexa.y[:,-1]

    # error
    err_qexa = np.abs(yqexa-ysol)
    norm_err_qexa = np.linalg.norm(err_qexa) / np.sqrt(nx)
    #print(f"Norme l2 de l'erreur pour dt = {dt:.3e} : {norm_err_qexa:.10e}")

    l_dt.append(dt)
    l_err_qexa.append(norm_err_qexa)

fig = go.Figure()
fig.add_trace(go.Scatter(x=l_dt, y=l_err_qexa, name='erreur'))
fig.add_trace(go.Scatter(x=l_dt, y=np.array(l_dt)**2, name='pente 2'))
fig.update_xaxes(type='log', exponentformat='e', title='dt')
fig.update_yaxes(type='log', exponentformat='e', title='erreur')
fig.update_layout(height=500)
fig.show()

## Méthode d'Euler implicite

In [ ]:
#############################################
def backward_euler(tini, tend, nt, yini, a):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = np.copy(yini)

    be_mat = sparse.eye(yini.size) - dt*a

    for it, tn  in enumerate(t[:-1]):
        yn = spsolve(be_mat.tocsr(), yn)

    return ode_result(t, yn)

### Solution

In [ ]:
d = 1.
xmin = -5.
xmax = 5.
nx = 1001
tini = 0.01
tend = 0.11

hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
dx = hm.dx
print('Dicrétisation spatiale : dx = ', dx)
x = np.linspace(xmin, xmax, nx)
fcn_exact = hm.fcn_exact
fcn = hm.fcn
a = hm.a

yini = fcn_exact(tini)
yexa = fcn_exact(tend)

sol_qexa = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=[tend], rtol=1e-12, atol=1.e-12)
yqexa = sol_qexa.y[:,0]

fig = make_subplots(rows=2, cols=1, subplot_titles=("Solution", "Erreur"), vertical_spacing=0.15)

fig.add_trace(go.Scattergl(x=x, y=yexa, name=f'sol exacte', legendgroup='sol', line_color='rgb(76,114,176)'), row=1, col=1)
fig.add_trace(go.Scattergl(x=x, y=yqexa, name=f'sol quasi-exacte', mode='lines+markers', legendgroup='sol'), row=1, col=1)

nt_vn = int(2*(tend-tini)/(dx*dx)) + 1
nt = [nt_vn-1800, nt_vn-1500, nt_vn, nt_vn+1000, nt_vn+3000]
for nt_i in nt:
    sol_num = backward_euler(tini, tend, nt_i, yini, a)
    ysol = sol_num.y
    err_exa  = np.abs(yexa-ysol)
    err_qexa = np.abs(yqexa-ysol)
    err_dx   = np.abs(yqexa-yexa)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=ysol, name=f'sol numérique', mode='lines+markers', legendgroup='sol', line_color='rgb(85,168,104)'), row=1, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_exa, name='err calculée à partir <br>  de la solution exacte', mode='lines+markers', legendgroup='err', line_color='rgb(76,114,176)'), row=2, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_qexa, name='err calculée à partir <br>  de la solution quasi-exacte', mode='lines+markers', legendgroup='err', line_color='rgb(221,132,82)'), row=2, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_dx, name='err en dx', mode='lines+markers', legendgroup='err', line_color='rgb(196,78,82)'), row=2, col=1)

for i in range(2+4,6+4): fig.data[i].visible = True

steps = []
for i, nt_i in enumerate(nt):
    args = [{"visible": [(el==0) or (el==1) or (el==4*i+2) or (el==4*i+3) or (el==4*i+4) or (el==4*i+5)  for el in range(len(fig.data))]}]
    #print(args)
    step = dict(method="update", label = f"{nt_i}", args=args)
    steps.append(step)
sliders = [dict(active=1, currentvalue={"prefix": "nt : "}, steps=steps)]

legend = dict(x=0.8, bgcolor='rgba(0,0,0,0)', tracegroupgap=280, groupclick='toggleitem')
fig.update_layout(height=800, sliders=sliders, legend=legend)
fig.update_yaxes(exponentformat='e')
fig.show()

### Ordre

In [ ]:
d = 1.
xmin = -5.
xmax = 5.
nx = 1001
tini = 0.01
tend = 0.11

hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
dx = hm.dx
print('Dicrétisation spatiale : dx = ', dx)
x = np.linspace(xmin, xmax, nx)
fcn_exact = hm.fcn_exact
fcn = hm.fcn
a = hm.a

yini = fcn_exact(tini)
yexa = fcn_exact(tend)

l_dt = []
l_err_qexa = [] 

for nt in [2001,  5001, 20001, 200001]:

    dt = (tend-tini)/(nt-1)

    sol = backward_euler(tini, tend, nt, yini, a)
    ysol = sol.y
    
    tol = 1.e-6
    t = np.linspace(tini, tend, nt)
    sol_qexa = solve_ivp(fcn, (tini, tend), yini, t_eval=t, method="Radau", rtol=tol, atol=1.e-3*tol)
    yqexa = sol_qexa.y[:,-1]

    # error
    err_qexa = np.abs(yqexa-ysol)
    norm_err_qexa = np.linalg.norm(err_qexa) / np.sqrt(nx)
    #print(f"Norme l2 de l'erreur pour dt = {dt:.3e} : {norm_err_qexa:.10e}")

    l_dt.append(dt)
    l_err_qexa.append(norm_err_qexa)

fig = go.Figure()
fig.add_trace(go.Scatter(x=l_dt, y=l_err_qexa, name='erreur'))
fig.add_trace(go.Scatter(x=l_dt, y=l_dt, name='pente 1'))
fig.update_xaxes(type='log', exponentformat='e', title='dt')
fig.update_yaxes(type='log', exponentformat='e', title='erreur')
fig.update_layout(height=500)
fig.show()

## Méthode de Crank-Nicolson

In [ ]:
#############################################
def crank_nicolson(tini, tend, nt, yini, a):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = np.copy(yini)

    cn_mat = sparse.eye(yini.size) - 0.5*dt*a

    for it, tn  in enumerate(t[:-1]):
        yn = spsolve(cn_mat.tocsr(), yn + 0.5*dt*a.dot(yn))

    return ode_result(t, yn)

### Solution

In [ ]:
d = 1.
xmin = -5.
xmax = 5.
nx = 1001
tini = 0.01
tend = 0.11

hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
dx = hm.dx
print('Dicrétisation spatiale : dx = ', dx)
x = np.linspace(xmin, xmax, nx)
fcn_exact = hm.fcn_exact
fcn = hm.fcn
a = hm.a

yini = fcn_exact(tini)
yexa = fcn_exact(tend)

sol_qexa = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=[tend], rtol=1e-12, atol=1.e-12)
yqexa = sol_qexa.y[:,0]

fig = make_subplots(rows=2, cols=1, subplot_titles=("Solution", "Erreur"), vertical_spacing=0.15)

fig.add_trace(go.Scattergl(x=x, y=yexa, name=f'sol exacte', legendgroup='sol', line_color='rgb(76,114,176)'), row=1, col=1)
fig.add_trace(go.Scattergl(x=x, y=yqexa, name=f'sol quasi-exacte', mode='lines+markers', legendgroup='sol'), row=1, col=1)

nt_vn = int(2*(tend-tini)/(dx*dx)) + 1
nt = [nt_vn-1800, nt_vn-1500, nt_vn, nt_vn+1000, nt_vn+3000]
for nt_i in nt:
    sol_num = crank_nicolson(tini, tend, nt_i, yini, a)
    ysol = sol_num.y
    err_exa  = np.abs(yexa-ysol)
    err_qexa = np.abs(yqexa-ysol)
    err_dx   = np.abs(yqexa-yexa)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=ysol, name=f'sol numérique', mode='lines+markers', legendgroup='sol', line_color='rgb(85,168,104)'), row=1, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_exa, name='err calculée à partir <br>  de la solution exacte', mode='lines+markers', legendgroup='err', line_color='rgb(76,114,176)'), row=2, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_qexa, name='err calculée à partir <br>  de la solution quasi-exacte', mode='lines+markers', legendgroup='err', line_color='rgb(221,132,82)'), row=2, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=x, y=err_dx, name='err en dx', mode='lines+markers', legendgroup='err', line_color='rgb(196,78,82)'), row=2, col=1)

for i in range(2+4,6+4): fig.data[i].visible = True

steps = []
for i, nt_i in enumerate(nt):
    args = [{"visible": [(el==0) or (el==1) or (el==4*i+2) or (el==4*i+3) or (el==4*i+4) or (el==4*i+5)  for el in range(len(fig.data))]}]
    #print(args)
    step = dict(method="update", label = f"{nt_i}", args=args)
    steps.append(step)
sliders = [dict(active=1, currentvalue={"prefix": "nt : "}, steps=steps)]

legend = dict(x=0.8, bgcolor='rgba(0,0,0,0)', tracegroupgap=280, groupclick='toggleitem')
fig.update_layout(height=800, sliders=sliders, legend=legend)
fig.update_yaxes(exponentformat='e')
fig.show()

### Ordre

In [ ]:
d = 1.
xmin = -5.
xmax = 5.
nx = 1001
tini = 0.01
tend = 0.11

hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
dx = hm.dx
print('Dicrétisation spatiale : dx = ', dx)
x = np.linspace(xmin, xmax, nx)
fcn_exact = hm.fcn_exact
fcn = hm.fcn
a = hm.a

yini = fcn_exact(tini)
yexa = fcn_exact(tend)

l_dt = []
l_err_qexa = [] 

for nt in [251, 501, 1001, 2001, 5001, 10001, 20001]: 

    dt = (tend-tini)/(nt-1)

    sol = crank_nicolson(tini, tend, nt, yini, a)
    ysol = sol.y
    
    tol = 1.e-9
    t = np.linspace(tini, tend, nt)
    sol_qexa = solve_ivp(fcn, (tini, tend), yini, t_eval=t, method="Radau", rtol=tol, atol=1.e-3*tol)
    yqexa = sol_qexa.y[:,-1]

    # error
    err_qexa = np.abs(yqexa-ysol)
    norm_err_qexa = np.linalg.norm(err_qexa) / np.sqrt(nx)
    #print(f"Norme l2 de l'erreur pour dt = {dt:.3e} : {norm_err_qexa:.10e}")

    l_dt.append(dt)
    l_err_qexa.append(norm_err_qexa)

fig = go.Figure()
fig.add_trace(go.Scatter(x=l_dt, y=l_err_qexa, name='erreur'))
fig.add_trace(go.Scatter(x=l_dt, y=np.array(l_dt)**2, name='pente 2'))
fig.update_xaxes(type='log', exponentformat='e', title='dt')
fig.update_yaxes(type='log', exponentformat='e', title='erreur')
fig.update_layout(height=500)
fig.show()